# Calcul du nombre de prélèvements Perchlorate non conforme par commune et par année

L'objectif de ce notebook est de partir de la liste de communes cog_communes, et pour chaque commune et chaque année, calculer le nombre de prélèvements non conformes pour le Perchlorate.

Pour créer ce notebook on a utilisé les codes de notebook cvm_calcul_conformite_par_commune.ipynb, sur ce lien https://github.com/dataforgoodfr/13_pollution_eau/pull/65

Il y aura plusieurs aggrégations à faire :

- commune (inseecommune) peut avoir plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être rattaché à plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être composé de plusieurs paramètres (cdparametresiseeaux) ; mais dans le cas du Perchlorate (comme CVM), il y a un seul paramètre selon la catégorisation de Pauline, donc c'est plus simple

La résultat final est dans la dernière cellule. Toutes les cellules précédentes sont là pour aider à la compréhension, en présentant des résultats intermédiaires.


In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%sqlcmd tables

Name
ana__resultats_communes
cog_communes
edc_communes
edc_prelevements
edc_resultats
laposte_communes
mapping_categories


In [3]:
%sql select * from cog_communes

Running query in 'duckdb:///../../database/data.duckdb'

TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT,de_partition,de_ingestion_date,de_dataset_datetime
COM,01001,84,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,None,2024,2025-03-07,20240220
COM,01002,84,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,None,2024,2025-03-07,20240220
COM,01004,84,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,None,2024,2025-03-07,20240220
COM,01005,84,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,None,2024,2025-03-07,20240220
COM,01006,84,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,None,2024,2025-03-07,20240220
COM,01007,84,01,01D,011,1,AMBRONAY,Ambronay,Ambronay,0101,None,2024,2025-03-07,20240220
COM,01008,84,01,01D,011,1,AMBUTRIX,Ambutrix,Ambutrix,0101,None,2024,2025-03-07,20240220
COM,01009,84,01,01D,011,1,ANDERT ET CONDON,Andert-et-Condon,Andert-et-Condon,0104,None,2024,2025-03-07,20240220
COM,01010,84,01,01D,011,1,ANGLEFORT,Anglefort,Anglefort,0110,None,2024,2025-03-07,20240220
COM,01011,84,01,01D,014,1,APREMONT,Apremont,Apremont,0114,None,2024,2025-03-07,20240220


In [4]:
%sql select *  from edc_communes

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,de_partition,de_ingestion_date,de_dataset_datetime
01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2020,2025-02-14,20230811-150005
01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2020,2025-02-14,20230811-150005
01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,2020,2025-02-14,20230811-150005
01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,2020,2025-02-14,20230811-150005
01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,2020,2025-02-14,20230811-150005
01005,AMBERIEUX-EN-DOMBES,Est,001000850,BDS CHATANIER,2023-01-01,2020,2025-02-14,20230811-150005
01005,AMBERIEUX-EN-DOMBES,Ouest,001000850,BDS CHATANIER,2021-08-01,2020,2025-02-14,20230811-150005
01006,AMBLEON,-,001000235,AMBLEON,2010-09-07,2020,2025-02-14,20230811-150005
01007,AMBRONAY,-,001000003,AMBRONAY,2010-09-07,2020,2025-02-14,20230811-150005
01008,AMBUTRIX,Ambutrix centre,001000254,AMBUTRIX MAIRIE,2010-09-07,2020,2025-02-14,20230811-150005


In [5]:
%sql select *  from edc_prelevements

Running query in 'duckdb:///../../database/data.duckdb'

cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date,de_dataset_datetime
001,001000003,01007,AMBRONAY,None,None,None,00100119766,2020-02-13,11h40,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100120290,2020-03-17,11h15,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100120717,2020-05-14,11h54,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100121072,2020-06-11,12h00,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100121547,2020-07-08,11h09,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100121589,2020-07-08,09h05,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100122035,2020-08-13,10h44,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100122549,2020-09-14,12h51,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100123096,2020-10-12,08h25,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100123173,2020-10-12,09h15,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005


In [6]:
%%sql 
SELECT categorie, COUNT(categorie) FROM mapping_categories
group by categorie

Running query in 'duckdb:///../../database/data.duckdb'

categorie,count(categorie)
perchlorate,1
phénol,23
non classé,139
cvm,1
métaux lourds,25
pesticides,674
métabolite de pesticide,127
médicament,5
hydrocarbure,163
dioxine et furane,17


In [7]:
%%sql 
SELECT * FROM mapping_categories
where categorie == 'perchlorate'

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
PCLAT,6219,PERCHLORATE,Perchlorate,14797-73-0,perchlorate


In [8]:
%%sql
    select *
    from "edc_resultats"
    where cdparametresiseeaux = 'PCLAT'
# on liste l'ensemble des analyses non conformes pour le paramètre PCLAT

Running query in 'duckdb:///../../database/data.duckdb'

cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
002,00200178364,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,<1,µg/L,133,None,None,0.0,14797-73-0,00200179474,2020,2025-02-14,20230811-150005
002,00200178377,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,"6,7",µg/L,133,None,None,6.7,14797-73-0,00200179487,2020,2025-02-14,20230811-150005
002,00200178394,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,<1,µg/L,133,None,None,0.0,14797-73-0,00200179504,2020,2025-02-14,20230811-150005
002,00200178471,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,<1,µg/L,133,None,None,0.0,14797-73-0,00200179581,2020,2025-02-14,20230811-150005
002,00200178481,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,"5,6",µg/L,133,None,None,5.6,14797-73-0,00200179591,2020,2025-02-14,20230811-150005
002,00200178489,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,<1,µg/L,133,None,None,0.0,14797-73-0,00200179599,2020,2025-02-14,20230811-150005
002,00200178496,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,<1,µg/L,133,None,None,0.0,14797-73-0,00200179606,2020,2025-02-14,20230811-150005
002,00200178505,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,<1,µg/L,133,None,None,0.0,14797-73-0,00200179615,2020,2025-02-14,20230811-150005
002,00200178508,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,<1,µg/L,133,None,None,0.0,14797-73-0,00200179618,2020,2025-02-14,20230811-150005
002,00200178518,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,"5,6",µg/L,133,None,None,5.6,14797-73-0,00200179628,2020,2025-02-14,20230811-150005


On constate qu'il n'y a pas de limite de qualité pour ce paramètre dans les données. ALors on a trouvé ces informations sur le site sante.gouv https://sante.gouv.fr/sante-et-environnement/eaux/article/perchlorates-dans-l-eau-du-robinet

- L’Anses dans son avis du 20 juillet 2012 conclut qu’au vu des données disponibles à ce jour, un dépassement modéré de la valeur de 15 µg/L chez l’adulte, notamment chez la femme enceinte, et de 4 µg/L chez le nouveau-né ne semble pas associé à des effets cliniquement décelables.
- On va prendre 15 µg/L comme limite de qualité.


In [9]:
%%sql
    select *
    from "edc_resultats"
    where cdparametresiseeaux = 'PCLAT' and valtraduite > 15

# on liste l'ensemble des analyses non conformes pour le paramètre Perchlorate

Running query in 'duckdb:///../../database/data.duckdb'

cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
002,00200178873,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,18,µg/L,133,None,None,18.0,14797-73-0,00200179983,2020,2025-02-14,20230811-150005
002,00200180674,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,23,µg/L,133,None,None,23.0,14797-73-0,00200181784,2020,2025-02-14,20230811-150005
002,00200181257,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,18,µg/L,133,None,None,18.0,14797-73-0,00200182367,2020,2025-02-14,20230811-150005
002,00200181325,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,32,µg/L,133,None,None,32.0,14797-73-0,00200182435,2020,2025-02-14,20230811-150005
002,00200181506,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,18,µg/L,133,None,None,18.0,14797-73-0,00200182616,2020,2025-02-14,20230811-150005
002,00200181697,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,18,µg/L,133,None,None,18.0,14797-73-0,00200182807,2020,2025-02-14,20230811-150005
002,00200181716,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,17,µg/L,133,None,None,17.0,14797-73-0,00200182826,2020,2025-02-14,20230811-150005
002,00200182038,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,45,µg/L,133,None,None,45.0,14797-73-0,00200183148,2020,2025-02-14,20230811-150005
002,00200182339,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,47,µg/L,133,None,None,47.0,14797-73-0,00200183449,2020,2025-02-14,20230811-150005
002,00200182417,PCLAT,6219,PERCHLORATE,Perchlorate,None,N,L,27,µg/L,133,None,None,27.0,14797-73-0,00200183527,2020,2025-02-14,20230811-150005


In [10]:
%%sql

with "resultats_pclat" as (
    select
      *,
      (CASE WHEN valtraduite > 15 THEN 1 ELSE 0 END) AS is_non_conforme
    from "edc_resultats"
    where cdparametresiseeaux = 'PCLAT'
),
"prelevements_pclat" as (
    select
        "cdreseau",
        "resultats_pclat"."de_partition",
        SUM(is_non_conforme) as "nbr_resultats_non_conformes",
        count(*) as "nbr_resultats_total",
        string_agg(CASE WHEN is_non_conforme = 1 THEN "resultats_pclat"."referenceprel" ELSE null END) as "list_referenceprels_non_conformes",
        string_agg(CASE WHEN is_non_conforme = 1 THEN "valtraduite" ELSE null END) as "list_valtraduite_non_conformes"
    from "resultats_pclat"
    left join "edc_prelevements" on
        "edc_prelevements"."referenceprel" = "resultats_pclat"."referenceprel"
        and
        "edc_prelevements"."de_partition" = "resultats_pclat"."de_partition"
    group by "cdreseau", "resultats_pclat"."de_partition"
)
select * from "prelevements_pclat" where "nbr_resultats_non_conformes" > 0

Running query in 'duckdb:///../../database/data.duckdb'

cdreseau,de_partition,nbr_resultats_non_conformes,nbr_resultats_total,list_referenceprels_non_conformes,list_valtraduite_non_conformes
080003918,2022,2,2,"08000114265,08000114852","25.0,25.0"
045000454,2020,1,1,04500143421,28.69
054001835,2020,1,4,05400144355,18.02
062000726,2020,1,1,06200241319,21.21
062000847,2020,1,1,06200242263,22.46
062000794,2023,1,3,06200277547,21.6
055000886,2023,2,5,"05500117766,05500118237","28.2,41.0"
062000585,2023,1,2,06200273853,33.55
080003916,2021,1,1,08000111087,25.0
062000714,2022,1,1,06200261953,29.29


In [11]:
%%sql

with "resultats_pclat" as (
    select
      *,
      (CASE WHEN valtraduite > 15 THEN 1 ELSE 0 END) AS is_non_conforme
    from "edc_resultats"
    where cdparametresiseeaux = 'PCLAT'
),
"prelevements_pclat" as (
    select
        "cdreseau",
        "resultats_pclat"."de_partition",
        SUM(is_non_conforme) as "nbr_resultats_non_conformes",
        count(*) as "nbr_resultats_total"
    from "resultats_pclat"
    left join "edc_prelevements" on
        "edc_prelevements"."referenceprel" = "resultats_pclat"."referenceprel"
        and
        "edc_prelevements"."de_partition" = "resultats_pclat"."de_partition"
    group by "cdreseau", "resultats_pclat"."de_partition"
),
"communes_pclat" as (
    select
        "inseecommune",
        "edc_communes"."de_partition",
        coalesce(sum("nbr_resultats_non_conformes"), 0) as "nbr_resultats_non_conformes",
        coalesce(sum("nbr_resultats_total"), 0) as "nbr_resultats_total",
        case
            when sum("nbr_resultats_non_conformes") > 0 then 'non conforme'
            when sum("nbr_resultats_total") > 0 then 'conforme'
            else 'non analysé'
        end as "resultat"
    from "edc_communes"
    left join "prelevements_pclat" on
        "prelevements_pclat"."cdreseau" = "edc_communes"."cdreseau"
        and
        "prelevements_pclat"."de_partition" = "edc_communes"."de_partition"
    group by "inseecommune", "edc_communes"."de_partition"
)
select * from communes_pclat where "nbr_resultats_non_conformes" > 0


Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,de_partition,nbr_resultats_non_conformes,nbr_resultats_total,resultat
59048,2022,2,3,non conforme
59552,2022,2,2,non conforme
62003,2022,1,1,non conforme
62570,2022,1,1,non conforme
62718,2022,2,3,non conforme
62873,2022,1,2,non conforme
80397,2022,2,2,non conforme
80685,2022,1,1,non conforme
02532,2023,1,1,non conforme
59092,2023,1,6,non conforme


In [12]:
%%sql

with "resultats_pclat" as (
    select
      *,
      (CASE WHEN valtraduite > 15 THEN 1 ELSE 0 END) AS is_non_conforme
    from "edc_resultats"
    where cdparametresiseeaux = 'PCLAT'
),
"prelevements_pclat" as (
    select
        "cdreseau",
        "resultats_pclat"."de_partition",
        SUM(is_non_conforme) as "nbr_resultats_non_conformes",
        count(*) as "nbr_resultats_total"
    from "resultats_pclat"
    left join "edc_prelevements" on
        "edc_prelevements"."referenceprel" = "resultats_pclat"."referenceprel"
        and
        "edc_prelevements"."de_partition" = "resultats_pclat"."de_partition"
    group by "cdreseau", "resultats_pclat"."de_partition"
),
"communes_pclat" as (
    select
        "inseecommune",
        "edc_communes"."de_partition",
        coalesce(sum("nbr_resultats_non_conformes"), 0) as "nbr_resultats_non_conformes",
        coalesce(sum("nbr_resultats_total"), 0) as "nbr_resultats_total",
        case
            when sum("nbr_resultats_non_conformes") > 0 then 'non conforme'
            when sum("nbr_resultats_total") > 0 then 'conforme'
            else 'non analysé'
        end as "resultat"
    from "edc_communes"
    left join "prelevements_pclat" on
        "prelevements_pclat"."cdreseau" = "edc_communes"."cdreseau"
        and
        "prelevements_pclat"."de_partition" = "edc_communes"."de_partition"
    group by "inseecommune", "edc_communes"."de_partition"
),
"annees" as (SELECT unnest(generate_series(2020, 2024)) as "annee")
select
    "cog"."COM" as "commune_code_insee",
    "cog"."LIBELLE" as "commune_nom",
    a."annee",
    coalesce("resultat", 'non analysé') as "resultat_pclat"
from "cog_communes" as "cog"
cross join 
    "annees" a
left join "communes_pclat" on
   "cog"."COM" = "communes_pclat"."inseecommune"
   and
   a."annee"::string =  "communes_pclat"."de_partition"


Running query in 'duckdb:///../../database/data.duckdb'

commune_code_insee,commune_nom,annee,resultat_pclat
01001,L'Abergement-Clémenciat,2020,non analysé
01002,L'Abergement-de-Varey,2020,non analysé
01004,Ambérieu-en-Bugey,2020,non analysé
01005,Ambérieux-en-Dombes,2020,non analysé
01006,Ambléon,2020,non analysé
01007,Ambronay,2020,non analysé
01008,Ambutrix,2020,non analysé
01009,Andert-et-Condon,2020,non analysé
01010,Anglefort,2020,non analysé
01011,Apremont,2020,non analysé
